In [1]:
import pandas as pd

ratings = pd.read_csv('/Users/Aman/Desktop/Mogen/ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
movies = pd.read_csv('/Users/Aman/Desktop/Mogen/ml-100k/u.item', sep='|', encoding='ISO-8859-1', names=['movie_id', 'title'], usecols=[0, 1])

# Merge ratings with movie titles fo r easier reference
ratings = ratings.merge(movies, on='movie_id')
print(ratings.head())

# Create the user-movie rating matrix
# Use pivot_table to handle duplicates by averaging the ratings
user_movie_matrix = ratings.pivot_table(index='user_id', columns='title', values='rating', aggfunc='mean').fillna(0)
print(user_movie_matrix.head())

FileNotFoundError: [Errno 2] No such file or directory: '/Users/Aman/Desktop/ml-100k/u.data'

In [6]:
from scipy.sparse.linalg import svds
import numpy as np

# Decompose the user-movie matrix with SVD
U, sigma, Vt = svds(user_movie_matrix, k=50)  # k is the number of latent features

# Convert sigma to a diagonal matrix
sigma = np.diag(sigma)

predicted_ratings = np.dot(np.dot(U, sigma), Vt)
predicted_ratings_df = pd.DataFrame(predicted_ratings, columns=user_movie_matrix.columns)
print(predicted_ratings_df.head())

title  'Til There Was You (1997)  1-900 (1994)  101 Dalmatians (1996)  \
0                      -0.022968     -0.036763               0.664976   
1                       0.090400      0.017455               0.616114   
2                      -0.001241      0.035042              -0.008985   
3                      -0.056875     -0.006834              -0.106642   
4                      -0.133296      0.145847               0.756527   

title  12 Angry Men (1957)  187 (1997)  2 Days in the Valley (1996)  \
0                 2.542402   -0.061299                     0.220617   
1                 0.394675    0.176855                     0.323936   
2                 0.246742    0.700922                    -0.118846   
3                -0.120627    0.388789                    -0.246415   
4                -0.216145   -0.061487                    -0.578149   

title  20,000 Leagues Under the Sea (1954)  2001: A Space Odyssey (1968)  \
0                                 1.438873                

In [ ]:
def recommend_movies(user_id, original_ratings, predicted_ratings_df, feedback=None, num_recommendations=5):
    """
    Recommend movies for a user based on their feedback and predicted ratings.
    """
    # Check if user_id is valid
    if user_id not in original_ratings['user_id'].unique():
        return ["Invalid user ID."]
    
    # Get already watched movies
    user_ratings = original_ratings[original_ratings['user_id'] == user_id]
    already_watched = user_ratings['title'].tolist()
    
    # Analyze feedback if provided
    filtered_movies = predicted_ratings_df.columns
    if feedback:
        genres = analyze_preferences(feedback)  # Uses Groq API
        genre_keywords = genres.split(", ")
        filtered_movies = [title for title in predicted_ratings_df.columns 
                           if any(genre.lower() in title.lower() for genre in genre_keywords)]
    
    # Get predicted ratings for the user
    user_predicted_ratings = predicted_ratings_df.loc[user_id - 1, filtered_movies].sort_values(ascending=False)
    
    # Recommend movies not already watched
    recommended_movies = [movie for movie in user_predicted_ratings.index if movie not in already_watched]
    return recommended_movies[:num_recommendations]

['E.T. the Extra-Terrestrial (1982)', 'Batman (1989)', 'Dave (1993)', "Ulee's Gold (1997)", "One Flew Over the Cuckoo's Nest (1975)"]
